# Computational Methods in Economics

## Lecture 3 - SLE - Solution to Classroom Exercises

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2019-12-04 13:15:58.430940


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import scipy.linalg

### Exercise 1

This question follows the example for the use of pivoting in Miranda and Fackler, section 2.3.

(a) Start with matrix 

\begin{equation}
    A = \left[\begin{array}{cc}
     -1\text{e-17} & 1 \\
     1 & 1
     \end{array}
     \right]
\end{equation}
 and vector 
\begin{equation}
    b = \left[\begin{array}{c}
     1 \\
     2 
     \end{array}
     \right]
\end{equation} 

Decompose $A$ analytically in matrices $L$ and $U$ using Gaussian elimination and confirm numerically that $A = L U$.

(b) Solve the system $Ax = b$ analytically using forward and backward substitution on $L$ and $U$ found in (a), and show that

\begin{equation}
    x_1 = \frac{1\text{e+17}}{1\text{e+17} + 1} \approx 1,\quad x_2 = \frac{1\text{e+17} + 2}{1\text{e+17} + 1} \approx 1
\end{equation} 

(c) Solve $Ax = b$ numerically using forward and backward substitution and compare the results. Note that you will need an implementation of backward substitution, for example from this week's problem set.

(d) Perform *pivoting* on the system: move greater elements to the diagonal by interchanging rows. In the system above, that gives matrix $\hat{A}$ and vector $\hat{b}$:

\begin{equation}
    \hat{A} = \left[\begin{array}{cc}
     1 & 1 \\
     -1\text{e-17} & 1
     \end{array}
     \right]
\end{equation}
 and vector 
\begin{equation}
    \hat{b} = \left[\begin{array}{c}
     2 \\
     1 
     \end{array}
     \right]
\end{equation} 

Verify that the system $\hat{A}x = \hat{b}$ has the same solution as the original system.

(e) As before, decompose $\hat{A}$ analytically in matrices $\hat{L}$ and $\hat{U}$ using Gaussian elimination and confirm numerically that $\hat{A} = \hat{L} \hat{U}$.

(f) Solve $\hat{A}x = \hat{b}$ numerically and verify that the result is close to the analytical solution found in (b).

In [3]:
def backward_sub(A, b):
    """
    Implements the backward-substitution algorithm to solve an upper triangular system of equations
    
    For doctest:
    
    >>> backward_sub( np.array([[1, 1], [0, 1]]), np.array([2, 1])  )
    array([ 1.,  1.])
    
    >>> backward_sub( np.array([[1, 2, 3], [0, 5, 7], [0, 0, 9]]), np.array([1, 2, 3])  )
    array([ 0.13333333, -0.06666667,  0.33333333])
    
     
    """
    ## check input: is A a square matrix?
    n, m = A.shape
    assert n == m, "A must be a square matrix"
    
    ## initialize solution vector
    x = np.zeros(n)
    
    ## fill solution vector using a for loop
    for i in range(n):
        
        ## compute sum on numerator of recursive rule
        summ = 0
        for j in range(i):
            summ += A[(n-1)-i, (n-1)-j] * x[(n-1)-j]
        
        ## use rule; NB: start at the last element in x!
        x[(n-1)-i] = (b[(n-1)-i] - summ) / A[(n-1)-i, (n-1)-i]
        
    return x     


def forward_sub(A, b):
    """
    Implements the forward-substitution algorithm to solve a lower triangular system of equations
    """
    n, m = A.shape   
    assert n == m, "A must be a square matrix"
    
    x = np.zeros(n)
    for i in range(n):
        
        summ = 0
        for j in range(i):
            summ += A[i, j] * x[j]
        
        x[i] = (b[i] - summ) / A[i, i]   
    
    return x

In [4]:
##### (a)

M = 1e+17
A = np.array([[-M**(-1), 1],
            [1, 1]])
b = np.array([1, 2])

print(A)

## results from analytical decomposition
L = np.array([[1, 0],
            [-M, 1]])
U = np.array([[-M**(-1), 1],
            [0, M + 1]])

print( L @ U )

[[-1.e-17  1.e+00]
 [ 1.e+00  1.e+00]]
[[-1.e-17  1.e+00]
 [ 1.e+00  0.e+00]]


In [5]:
##### (c)

## solve system numerically using backward and forward substitution
print( backward_sub(U, forward_sub(L, b)) )

[-0.  1.]


In [6]:
##### (e) 

## system with pivoting
A_hat = np.array([[1, 1],
            [-M**(-1), 1]])
b_hat = np.array([2, 1])

print(A_hat)

L_hat = np.array([[1, 0],
            [-M**(-1), 1]])
U_hat = np.array([[1, 1],
            [0, M**(-1) + 1]])
print( L_hat @ U_hat )

[[ 1.e+00  1.e+00]
 [-1.e-17  1.e+00]]
[[ 1.e+00  1.e+00]
 [-1.e-17  1.e+00]]


In [7]:
##### (f)

## solve system numerically using backward and forward substitution
backward_sub(U_hat, forward_sub(L_hat, b_hat))

array([1., 1.])

We can show that pivoting is used by Scipy's **linalg.solve** algorithm. First note that **linalg.solve** gives the correct solution when using the *original* system **Ax = b**:  

In [8]:
scipy.linalg.solve(A, b)

array([1., 1.])

In the background, the algorithm applies pivoting before decomposing the matrix **A**. We can see this by using **linalg.lu** that implements LU factorization. It returns three outputs **P**, **L_lu** and **U_lu**. The product **P @ L_lu** gives a permutable lower triangular matrix. 

The resulting matrices **L_lu** und **U_lu** are the same as our analytically obtained matrices **L_hat** and **U_hat**: 

In [9]:
P, L_lu, U_lu = scipy.linalg.lu(A)
print(L_lu == L_hat)

[[ True  True]
 [ True  True]]


In [10]:
print(U_lu == U_hat)

[[ True  True]
 [ True  True]]


Note that multiplying **L_lu @ U_lu** gives **A_hat**, while multiplying **P @ L_hat @ U_hat** gives the original matrix **A**:

In [11]:
print( L_hat @ U_hat == A_hat)

[[ True  True]
 [ True  True]]


In [12]:
print( P @ L_hat @ U_hat == A)

[[ True  True]
 [ True  True]]


### Exercise 2

Find the solution to the following SLE using a) Gauss-Jacobi and b) Gauss-Seidel. Which one is faster, i.e. needs less iterations? 

In [13]:
A = np.array([[54, 14, -11, 2], 
              [14, 50, -4, 29],
              [-11, -4, 55, 22],
              [2, 29, 22, 95]]
            )
b = np.array([1, 1, 1, 1])

In [14]:
#### (a) Gauss-Jacobi

def gauss_jacobi(A, b, x0):
    """ 
    Implements the Gauss-Seidel method with a over-relaxation parameter
    """
    eps = 1
    tol = 1e-8
    it = 0
    maxit = 100

    x = x0
    Q = np.diag(np.diag(A))
    Q_inv = np.linalg.inv(Q)
    
    while eps > tol and it < maxit:
        it += 1 
        x_new = Q_inv @ b + ( np.eye(len(b)) - Q_inv @ A) @ x
        eps = np.linalg.norm(x_new - x)
        x = x_new
        
        
    return x, it 

In [15]:
%%timeit -n1 -r1
x0 = np.zeros(4)
gauss_jacobi(A, b, x0)
x, it = gauss_jacobi(A, b, x0)
print('Number of iterations = {}'.format(it))
print(x)

Number of iterations = 25
[ 0.01893441  0.01680507  0.02335523 -0.00041085]
8.91 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
#### (b) Gauss-Seidel 

def gauss_seidel(A, b, x0):
    """ 
    Implements the Gauss-Seidel method with a over-relaxation parameter
    """
    eps = 1
    tol = 1e-8
    it = 0
    maxit = 100

    x = x0
    Q = np.triu(A)
    Q_inv = np.linalg.inv(Q)
    
    while eps > tol and it < maxit:
        it += 1 
        x_new = Q_inv @ b + ( np.eye(len(b)) - Q_inv @ A) @ x
        eps = np.linalg.norm(x_new - x)
        x = x_new
        
    return x, it  

In [17]:
%%timeit -n1 -r1
x0 = np.zeros(4)
gauss_seidel(A, b, x0) 
x, it = gauss_seidel(A, b, x0)
print('Number of iterations = {}'.format(it))
print(x)

Number of iterations = 14
[ 0.01893441  0.01680508  0.02335523 -0.00041085]
4.03 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
